Upload functions and necessary modules

In [1]:
from bb2022_functions import *
%matplotlib inline
from Bio.SeqIO.FastaIO import SimpleFastaParser
from Bio import SeqIO
pd.options.mode.chained_assignment = None  # default='warn'

Import and format metadata

In [2]:
md = pd.read_csv("metadata_merged.csv")
merged = pd.read_csv("metadata_niskin.csv")
all_md = pd.read_csv("allmetadata.csv")

Add microbial communities

In [6]:
#generate a dataframe from all specified amplicon
df, comm = consolidate_tables('16S')
merged = merge_metadata(df, all_md)
separated = pick_metadata(merged)

Found all 16S tables.
Successfully saved all tables.
Set up metadata ...
Saved merged_asvs_metadata.tsv
Appended all taxonomies to taxos
Saved separated by metadata dataframe.


In [7]:
searchfor = ["Cyanobacteria", "Chloroplast"]
contaminants = separated[separated.Taxon.str.contains('|'.join(searchfor))]
separated = separated[~separated.Taxon.str.contains('|'.join(searchfor))]
separated = separated.reset_index(drop=True)

In [8]:
#run these lines to switch to chloroplast comm
separated = contaminants.copy()
separated = separated.reset_index(drop=True)
comm = 'chloroplast'

#add phytorep taxonomy
cp_tax = pd.read_csv('chloroplast/taxonomy.tsv', sep='\t')
cp_tax = cp_tax.rename(columns={"Feature ID": "feature_id", "Taxon": "PRTaxon", "Confidence":"PRConfidence"})
separated = pd.merge(separated, cp_tax, on="feature_id", how="left")
separated['PRSpecies'] = separated['PRTaxon'].str.split('|').str[-1]

In [9]:
#re-calculate ratios when removing chloroplast for 16S
separated['Total'] = separated['feature_frequency'].groupby(separated['sampleid']).transform('sum')
separated['ratio'] = separated['feature_frequency']/separated['Total']
separated['nASVs'] = separated['feature_id'].groupby(separated['sampleid']).transform('count')
separated['weekdepth'] = separated["weekn"].astype(str) + separated["depth"].astype(str)
separated['avg'] = separated['nASVs'].groupby(separated['weekdepth']).transform('mean')
separated['diff'] = separated['nASVs'] - separated['avg']

In [10]:
newseparated = make_defract(all_md, separated)
newseparated["rank"] = newseparated.groupby("sampleid")["ratio"].rank(method="average", ascending=False)
newseparated["ranktot"] = newseparated['rank'] / newseparated['nASVs']